In [1]:
import os
import sys
import glob
import numpy as np
import pandas as pd
from IPython.display import display
import time 
import tracker
latest_date = tracker.processing_dates[-1]
os.chdir('/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('max_colwidth', 400)

In [2]:
latest_date

'2022.09.22.15.30'

In [3]:
# setting input and output with jupyter notebook in context 
if 'ipykernel_launcher.py' in sys.argv[0]:
    input_fn = 'results/samplesheets/hicpro/current.hicpro.samplesheet.without_header.tsv'.format(latest_date)
    output_prefix = 'results/samplesheets/post-hicpro/{}.post-hicpro.post-check.fithichip-loop-calling'.format(latest_date)
else:
    input_fn = sys.argv[1]
    output_fn = sys.argv[2]

## Helper Functions

In [4]:
def read_out_log(log):
    
    status = 0
    f_status = 0
    sample_name = 'check'
    peaks_found = 0
    no_interact = 0
    date = ''

    # extract job_id and index
    meta = os.path.basename(log)
    meta = meta.split('.')[1].split('-')
    job_id = meta[0].replace('o', '')
    index = meta[1]

    with open(log, errors='ignore') as fr:
        for line in fr:
            info = line.strip()
            if 'sample_name' in info:
                sample_name = info.split()[1]
            if 'chip-seq peaks found and will be used to call loops' in info:
                peaks_found = 1
            if 'FitHiChIP pipeline is completely executed - congratulations !!!' in info:
                f_status = 1
            if 'SORRY !!!!!!!! FitHiChIP could not find any statistically significant interactions' in info:
                no_interact = 1
            if 'Ended: fithichip loop calling' in info:
                status = 1
                

    return([index, sample_name, job_id, peaks_found, no_interact, f_status, status])

In [5]:
def read_error_log(log):
    
    status = 1
    eline = []
    has_errors = 0
    mem = 0
    
    with open(log, errors='ignore') as fr:
        for line in fr:
            info = line.strip()
            if 'cat: /var/spool/torque/aux' in info:
                continue
            if 'Loading required package:' in info:
                continue
            if 'Attaching package:' in info:
                continue
            if 'The following objects are masked' in info:
                continue
            if line.startswith('\t'):
                continue
            if 'The following object is masked' in info:
                continue
            if 'spar-finding: non-finite value inf; using BIG value' in info:
                continue
            if 'Warning message' in info:
                continue
            if line.startswith('\n'):
                continue
            if 'cluster' in line:
                continue
            if 'par' in line:
                continue
            if 'IQR' in line or 'anyDuplicated' in line or 'dirname' in line or 'grepl' in line or 'order' in line or 'rbind' in line or 'union' in line or 'expand.grid' in line or 'shift' in line or 'first, second' in line:
                continue
            if 'containing missing values (geom_bar).' in line:
                continue
            if 'containing missing values (geom_path).' in line:
                continue
            if 'containing missing values (geom_point).' in line:
                continue
            if 'NaNs produced' in line:
                continue
            if 'In data.frame(group = paste(' in line:
                continue
            status = 0
            eline.append(line)
            if 'memory' in line:
                mem = 1
        if len(eline) != 0:
            has_errors = 1
            
    return(status, eline, has_errors, mem)

## Check Logs

In [6]:
# load the data
df = pd.read_table(input_fn, skiprows=0)
df['sample_index'] = df.index
print('number of samples:', len(df))

number of samples: 485


In [7]:
df

,293T.GSE128106.Homo_Sapiens.YY1.b1,GSE128106,Homo_Sapiens,1,YY1,HindIII,293T,sample_index
0,293T-PDS.GSE128106.Homo_Sapiens.YY1.b1,GSE128106,Homo_Sapiens,1,YY1,HindIII,293T-PDS,0
1,293T-TMPYP4.GSE128106.Homo_Sapiens.YY1.b1,GSE128106,Homo_Sapiens,1,YY1,HindIII,293T-TMPYP4,1
2,A673_SA1m1.GSE133227.Homo_Sapiens.CTCF.b1,GSE133227,Homo_Sapiens,1,CTCF,MboI,A673_SA1m1,2
3,A673_SA1m1.GSE133227.Homo_Sapiens.CTCF.b2,GSE133227,Homo_Sapiens,2,CTCF,MboI,A673_SA1m1,3
4,A673_SA2m1.GSE133227.Homo_Sapiens.CTCF.b1,GSE133227,Homo_Sapiens,1,CTCF,MboI,A673_SA2m1,4
5,A673_SA2m1.GSE133227.Homo_Sapiens.CTCF.b2,GSE133227,Homo_Sapiens,2,CTCF,MboI,A673_SA2m1,5
6,A673_SA2m1.GSE133227.Homo_Sapiens.CTCF.b3,GSE133227,Homo_Sapiens,3,CTCF,MboI,A673_SA2m1,6
7,A673_SA2m1.GSE133227.Homo_Sapiens.CTCF.b4,GSE133227,Homo_Sapiens,4,CTCF,MboI,A673_SA2m1,7
8,A673_SA2m1.GSE133227.Homo_Sapiens.CTCF.b5,GSE133227,Homo_Sapiens,5,CTCF,MboI,A673_SA2m1,8
9,A673_SA2m1.GSE133227.Homo_Sapiens.H3K27ac.b1,GSE133227,Homo_Sapiens,1,H3K27ac,MboI,A673_SA2m1,9


In [8]:
# create a dataframe of log information, 506307, 508200, 5085036 (fixing mem errors), 50850 37-42 (hichip-peaks), 51014 (fithichip peaks)
output_logs = glob.glob('results/loops/logs/run_fithichip_loopcalling*.o5345*-*')
log_data = []   
for out_log in output_logs: 
    sample_index = out_log.split('-')[1]
    index, sample_name, job_id, peaks_found, no_interact, f_status, status = read_out_log(out_log)
    error_logs = [log for log in glob.glob('results/loops/logs/run_fithichip_loopcalling*.e5345*-*') if log.split('_')[3].split('.')[0] == out_log.split('/')[3].split('_')[3].split('.')[0] and log.split('-')[1] == sample_index]
    error_log = error_logs[0]
    if len(error_logs) != 1:
        print("Length more than 1")
        print(error_logs)
    error_status, eline, has_errors, mem = read_error_log(error_log)
    log_data.append([index, sample_name, job_id, peaks_found, no_interact, f_status, status, error_status, eline, has_errors, mem, os.path.basename(out_log)])
log_df = pd.DataFrame(log_data)
log_df.columns = ['sample_index', 'std_sample_name', 'job_id', 'peaks_found', 'no_interact', 'f_status', 'out_status', 'error_status', 'errors', 'has_errors', 'mem', 'log']
log_df.drop('sample_index', inplace=True, axis=1)
log_df = log_df.sort_values(by=["std_sample_name"], ascending=True)

In [9]:
log_df = log_df.sort_values(by=["std_sample_name"], ascending=True).reset_index(drop=True)
log_df

,std_sample_name,job_id,peaks_found,no_interact,f_status,out_status,error_status,errors,has_errors,mem,log
0,CD4_Naive_1800-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b1,5345875,0,0,1,1,0,"[Error in (function (file = if (onefile) ""Rplots.pdf"" else ""Rplot%03d.pdf"", : \n, cannot open file 'Rplots.pdf'\n, Calls: <Anonymous> -> print.ggplot -> grid.newpage -> <Anonymous>\n, Execution halted\n]",1,0,run_fithichip_loopcalling_L25.o5345875-373
1,CD4_Naive_1800-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b1,5345878,0,0,1,1,0,"[Error in (function (file = if (onefile) ""Rplots.pdf"" else ""Rplot%03d.pdf"", : \n, cannot open file 'Rplots.pdf'\n, Calls: <Anonymous> -> print.ggplot -> grid.newpage -> <Anonymous>\n, Execution halted\n]",1,0,run_fithichip_loopcalling_S25.o5345878-373
2,CD4_Naive_1800-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b1,5345874,0,0,1,1,0,"[Error in (function (file = if (onefile) ""Rplots.pdf"" else ""Rplot%03d.pdf"", : \n, cannot open file 'Rplots.pdf'\n, Calls: <Anonymous> -> print.ggplot -> grid.newpage -> <Anonymous>\n, Execution halted\n]",1,0,run_fithichip_loopcalling_L10.o5345874-373
3,CD4_Naive_1800-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b1,5345873,0,0,1,1,0,"[Error in (function (file = if (onefile) ""Rplots.pdf"" else ""Rplot%03d.pdf"", : \n, cannot open file 'Rplots.pdf'\n, Calls: <Anonymous> -> print.ggplot -> grid.newpage -> <Anonymous>\n, Execution halted\n]",1,0,run_fithichip_loopcalling_L5.o5345873-373
4,CD4_Naive_1800-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b1,5345876,0,0,1,1,0,"[Error in (function (file = if (onefile) ""Rplots.pdf"" else ""Rplot%03d.pdf"", : \n, cannot open file 'Rplots.pdf'\n, Calls: <Anonymous> -> print.ggplot -> grid.newpage -> <Anonymous>\n, Execution halted\n]",1,0,run_fithichip_loopcalling_S5.o5345876-373
5,CD4_Naive_1800-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b1,5345877,0,0,1,1,0,"[Error in (function (file = if (onefile) ""Rplots.pdf"" else ""Rplot%03d.pdf"", : \n, cannot open file 'Rplots.pdf'\n, Calls: <Anonymous> -> print.ggplot -> grid.newpage -> <Anonymous>\n, Execution halted\n]",1,0,run_fithichip_loopcalling_S10.o5345877-373
6,CD4_Naive_1800-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b2,5345873,0,0,1,1,0,"[Error in (function (file = if (onefile) ""Rplots.pdf"" else ""Rplot%03d.pdf"", : \n, cannot open file 'Rplots.pdf'\n, Calls: <Anonymous> -> print.ggplot -> grid.newpage -> <Anonymous>\n, Execution halted\n]",1,0,run_fithichip_loopcalling_L5.o5345873-379
7,CD4_Naive_1800-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b2,5345875,0,0,1,1,0,"[Error in (function (file = if (onefile) ""Rplots.pdf"" else ""Rplot%03d.pdf"", : \n, cannot open file 'Rplots.pdf'\n, Calls: <Anonymous> -> print.ggplot -> grid.newpage -> <Anonymous>\n, Execution halted\n]",1,0,run_fithichip_loopcalling_L25.o5345875-379
8,CD4_Naive_1800-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b2,5345874,0,0,1,1,0,"[Error in (function (file = if (onefile) ""Rplots.pdf"" else ""Rplot%03d.pdf"", : \n, cannot open file 'Rplots.pdf'\n, Calls: <Anonymous> -> print.ggplot -> grid.newpage -> <Anonymous>\n, Execution halted\n]",1,0,run_fithichip_loopcalling_L10.o5345874-379
9,CD4_Naive_1800-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b2,5345878,0,0,1,1,0,"[Error in (function (file = if (onefile) ""Rplots.pdf"" else ""Rplot%03d.pdf"", : \n, cannot open file 'Rplots.pdf'\n, Calls: <Anonymous> -> print.ggplot -> grid.newpage -> <Anonymous>\n, Execution halted\n]",1,0,run_fithichip_loopcalling_S25.o5345878-379


## Identifying problem samples

In [10]:
problems = (log_df.mem == 1)
problems_df = log_df.loc[problems,:]
print('number of problem samples:', len(problems_df))

number of problem samples: 0


In [21]:
problems_df.loc[:, ['std_sample_name', 'job_id', 'peaks_found', 'f_status', 'out_status', 'no_interact', 'error_status', 'errors', 'log']]

,std_sample_name,job_id,peaks_found,f_status,out_status,no_interact,error_status,errors,log
108,Erythroid_0hr.GSE142004.Mus_Musculus.YY1.b1,5244802,1,0,1,1,0,"[Error: no function to return from, jumping to top level\n, Execution halted\n, awk: fatal: cannot open file `/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/loops/fithichip/Erythroid_0hr.GSE142004.Mus_Musculus.YY1.b1_chipseq.peaks/S25//FitHiChIP_Peak2ALL_b25000_L20000_U2000000/P2PBckgr_1/Coverage_Bias/FitHiC_BiasCorr/FitHiChIP-S25.interactions_FitHiC.bed' for reading (N...",run_fithichip_loopcalling_S25.o5244802-83
109,Erythroid_0hr.GSE142004.Mus_Musculus.YY1.b1,5244800,1,0,1,1,0,"[Error in lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) : \n, 0 (non-NA) cases\n, Calls: as.data.frame -> lapply -> FUN -> lm -> lm.fit\n, Execution halted\n, awk: fatal: cannot open file `/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/loops/fithichip/Erythroid_0hr.GSE142004.Mus_Musculus.YY1.b1_chipseq.peaks/S5//FitHiChIP_Peak2ALL_b5000_L20000_U2000000...",run_fithichip_loopcalling_S5.o5244800-83
110,Erythroid_0hr.GSE142004.Mus_Musculus.YY1.b1,5244797,1,0,1,1,0,"[Error in (function (file = if (onefile) ""Rplots.pdf"" else ""Rplot%03d.pdf"", : \n, cannot open file 'Rplots.pdf'\n, Calls: <Anonymous> -> print.ggplot -> grid.newpage -> <Anonymous>\n, Execution halted\n]",run_fithichip_loopcalling_L5.o5244797-83
111,Erythroid_0hr.GSE142004.Mus_Musculus.YY1.b1,5244801,1,0,1,1,0,"[Error: no function to return from, jumping to top level\n, Execution halted\n, awk: fatal: cannot open file `/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/loops/fithichip/Erythroid_0hr.GSE142004.Mus_Musculus.YY1.b1_chipseq.peaks/S10//FitHiChIP_Peak2ALL_b10000_L20000_U2000000/P2PBckgr_1/Coverage_Bias/FitHiC_BiasCorr/FitHiChIP-S10.interactions_FitHiC.bed' for reading (N...",run_fithichip_loopcalling_S10.o5244801-83
112,Erythroid_0hr.GSE142004.Mus_Musculus.YY1.b1,5244798,1,0,1,1,0,"[Error in (function (file = if (onefile) ""Rplots.pdf"" else ""Rplot%03d.pdf"", : \n, cannot open file 'Rplots.pdf'\n, Calls: <Anonymous> -> print.ggplot -> grid.newpage -> <Anonymous>\n, Execution halted\n]",run_fithichip_loopcalling_L10.o5244798-83
113,Erythroid_0hr.GSE142004.Mus_Musculus.YY1.b1,5244799,1,0,1,1,0,"[Error in (function (file = if (onefile) ""Rplots.pdf"" else ""Rplot%03d.pdf"", : \n, cannot open file 'Rplots.pdf'\n, Calls: <Anonymous> -> print.ggplot -> grid.newpage -> <Anonymous>\n, Execution halted\n]",run_fithichip_loopcalling_L25.o5244799-83
114,Erythroid_0hr.GSE142004.Mus_Musculus.YY1.b2,5244800,1,0,1,1,0,"[Error in smooth.spline(avg_int_dist[non_NA_idx], prior_contact_prob[non_NA_idx], : \n, need at least four unique 'x' values\n, Execution halted\n, awk: fatal: cannot open file `/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/loops/fithichip/Erythroid_0hr.GSE142004.Mus_Musculus.YY1.b2_chipseq.peaks/S5//FitHiChIP_Peak2ALL_b5000_L20000_U2000000/P2PBckgr_1/Coverage_Bias/...",run_fithichip_loopcalling_S5.o5244800-84
115,Erythroid_0hr.GSE142004.Mus_Musculus.YY1.b2,5244797,1,0,1,1,0,"[Error in (function (file = if (onefile) ""Rplots.pdf"" else ""Rplot%03d.pdf"", : \n, cannot open file 'Rplots.pdf'\n, Calls: <Anonymous> -> print.ggplot -> grid.newpage -> <Anonymous>\n, Execution halted\n]",run_fithichip_loopcalling_L5.o5244797-84
116,Erythroid_0hr.GSE142004.Mus_Musculus.YY1.b2,5244798,1,0,1,1,0,"[Error in (function (file = if (onefile) ""Rplots.pdf"" else ""Rplot%03d.pdf"", : \n, cannot open file 'Rplots.pdf'\n, Calls: <Anonymous> -> print.ggplot -> grid.newpage -> <Anonymous>\n, Execution halted\n]",run_fithichip_loopcalling_L10.o5244798-84
117,Erythroid_0hr.GSE142004.Mus_Musculus.YY1.b2,5244801,1,0,1,1,0,"[Error in lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...) : \n, 0 (non-NA) cases\n, Calls: as.data.frame -> lapply -> FUN -> lm -> lm.fit\n, Execution halted\n, awk: fatal: cannot open file `/mnt/bioadhoc-t

In [35]:
problems_df = problems_df.sort_values(by='std_sample_name').reset_index(drop=True)
problems_df.to_excel("results/loops/problems.xlsx", sheet_name='sample_summary')